<a href="https://colab.research.google.com/github/ShanshanHoo/Brain-Tumour-Segmentation/blob/master/RAG_Cobenfy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai scikit-learn numpy sentence-transformers PyMuPDF langchain ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from google.colab import userdata

from openai import OpenAI
import fitz
from sentence_transformers import SentenceTransformer
import openai
import sqlite3
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.text_splitter import RecursiveCharacterTextSplitter # import the missing library


In [ ]:
# Set up OpenAI API
openai.api_key=userdata.get('OPENAI_API_KEY')

# Load multilingual embedding model (E5-small model)
model = SentenceTransformer('intfloat/multilingual-e5-small')

# Database setup (SQLite for simplicity)
conn = sqlite3.connect('embeddings.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS embeddings
              (chunk_id INTEGER PRIMARY KEY, text_chunk TEXT, embedding BLOB)''') #BLOB (Binary Large Object)
conn.commit()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:

# Function to read PDF and split into chunks
def read_pdf(file_path, chunk_size=500):
    doc = fitz.open(file_path)
    concatenated_text = ""
    for page in doc:
      text=page.get_text()
      concatenated_text+=text

    # Split text into chunks
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)
    chunks=text_splitter.split_text(concatenated_text)

    return chunks

# Function to embed chunks and store in the database
def embed_and_store_chunks(chunks):
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk)
        # Store embedding and chunk in the database (BLOB format for embedding)
        cursor.execute("INSERT INTO embeddings (text_chunk, embedding) VALUES (?, ?)",
                       (chunk, embedding.tobytes()))
    conn.commit()

from sklearn.preprocessing import normalize

def retrieve_relevant_chunks(query, top_n=5):
    # Encode the query
    query_embedding = model.encode(query)

    # Normalize the query embedding
    query_embedding = normalize([query_embedding])[0]

    # Retrieve all embeddings from the database
    cursor.execute("SELECT text_chunk, embedding FROM embeddings")
    all_embeddings = cursor.fetchall()

    similarities = []
    for text_chunk, embedding in all_embeddings:
        embedding_np = np.frombuffer(embedding, dtype=np.float32)
        # Normalize the chunk embedding
        embedding_np = normalize([embedding_np])[0]
        similarity = cosine_similarity([query_embedding], [embedding_np])[0][0]
        similarities.append((similarity, text_chunk))

    similarities.sort(reverse=True, key=lambda x: x[0])
    return [chunk for _, chunk in similarities[:top_n]]

# Function to generate a response using GPT-4 with the retrieved context
def generate_gpt4_response(query, relevant_chunks):
    # Combine the relevant chunks to form the context
    client = OpenAI(api_key=openai.api_key)
    context = "\n".join(relevant_chunks)
    prompt = f"Based on the following context:\n\n{context}\n\nAnswer the query: {query}"

    # Call GPT-4o with the context
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
  ],
        max_tokens=500,
        temperature=0.7
    )

    return response.choices[0].message.content



In [ ]:
# Example usage
if __name__ == "__main__":
    # Step 1: Read and process the PDF file
    pdf_file_path = '/content/Cobenfy_Drug.com.pdf'  # Replace with your PDF path
    pdf_chunks= read_pdf(pdf_file_path)

    # Step 2: Embed chunks and store them in the database
    embed_and_store_chunks(pdf_chunks)

    # Step 3: Query the system and retrieve relevant chunks
    #user_query = "Could you provide a short summary of this new drug?"  # Replace with your query
    #user_query = "Based on provided documents, what's the recommended dosage for patients just started with Cobenfy?"
    user_query = "Based on provided documents, what's the active ingredients of Cobenfy?"
    relevant_chunks = retrieve_relevant_chunks(user_query)
    #print(relevant_chunks)

    # Step 4: Use GPT-4 to generate an augmented response using the relevant chunks
    gpt4_response = generate_gpt4_response(user_query, relevant_chunks)

    # Output the final response
    print("GPT-4 Response:", gpt4_response)

GPT-4 Response: The active ingredients of Cobenfy are xanomeline and trospium chloride.


In [ ]:
client = OpenAI(api_key=openai.api_key)
response_raw = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_query}
  ],
        max_tokens=500,
        temperature=0.7
    )
response_raw

ChatCompletion(id='chatcmpl-BLJT08vAvM7g4vQPUtpZbgwTR98p2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm sorry, but I don't have access to external documents or databases to look up specific products like Cobenfy. However, I can help you find information if you provide details about the product or its intended use. Alternatively, you might consider checking the packaging or the manufacturer's website for the active ingredients.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744419418, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_44added55e', usage=CompletionUsage(completion_tokens=61, prompt_tokens=31, total_tokens=92, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(

In [ ]:
print( pdf_chunks)

['Cobenfy\nPronunciation: co-BEN-fee\nGeneric name: xanomeline and trospium chloride\nDosage form: capsules (50mg/20mg, 100mg/20mg, 125mg/30mg) xanomeline/trospium\nWhat is Cobenfy?\nCobenfy (xanomeline and trospium chloride) is a new treatment for schizophrenia used to improve\npositive and negative symptoms. Cobenfy contains xanomeline, which helps schizophrenia symptoms,\nand trospium, which works to reduce the side effects. Cobenfy is a new class of medicine and works', 'positive and negative symptoms. Cobenfy contains xanomeline, which helps schizophrenia symptoms,\nand trospium, which works to reduce the side effects. Cobenfy is a new class of medicine and works\ndifferently than traditional schizophrenia medications so it does not have a boxed warning and atypical\nantipsychotic class warnings and precautions.\nSchizophrenia is related to imbalances in messenger systems in the brain involving muscarinic', 'antipsychotic class warnings and precautions.\nSchizophrenia is related t

In [ ]:
query_embedding = model.encode( "Based on provided documents, what's the active ingredients of Cobenfy?")
query_embedding

In [ ]:
# More Questions
relevant_chunks

['Cobenfy ingredients\xa0\nActive ingredients:\u200a xanomeline and trospium chloride\xa0\nInactive ingredients: ascorbic acid, lactose monohydrate, microcrystalline cellulose, and talc Capsule\nshell: contains black iron oxide (only100mg/20mg), hypromellose, red iron oxide, titanium dioxide, and\nyellow iron oxide (only 50mg/20mg and 100mg/20mg)\nCompany\nCobenfy Bristol-Myers Squibb Company Princeton, NJ 08543 USA.\nReferences',
 'positive and negative symptoms. Cobenfy contains xanomeline, which helps schizophrenia symptoms,\nand trospium, which works to reduce the side effects. Cobenfy is a new class of medicine and works\ndifferently than traditional schizophrenia medications so it does not have a boxed warning and atypical\nantipsychotic class warnings and precautions.\nSchizophrenia is related to imbalances in messenger systems in the brain involving muscarinic',
 "receptors. Cobenfy's active ingredient tiospium chloride's MOA is a muscarinic antagonist that blocks\nthe muscarin

## RAGas

Evaluation of Rag Metric

In [ ]:
# Make sure four components are matched
from datasets import Dataset
questions = ["What are the active ingredients in Cobenfy?"]
ground_truths = ["Cobenfy contains xanomeline and trospium chloride. Xanomeline acts on brain receptors, while trospium reduces peripheral side effects."]
answers = [gpt4_response]
contexts = [relevant_chunks]
# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)


In [ ]:
import os
from ragas import evaluate
from google.colab import userdata
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

# Get your API key from Colab secret manager
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
 # Manage the API key using the context manager
result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)


df = result.to_pandas()
df

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What are the active ingredients in Cobenfy?,[Cobenfy ingredients \nActive ingredients: xa...,The active ingredients of Cobenfy are xanomeli...,Cobenfy contains xanomeline and trospium chlor...,0.8875,1.0,1.0,0.998683


In [ ]:
# Try three question sets
questions = [
    "What are the active ingredients in Cobenfy?",
    "How should Cobenfy be taken?",
    "Who should not take Cobenfy?",
]

ground_truths = [
    "Cobenfy contains xanomeline and trospium chloride. Xanomeline acts on brain receptors, while trospium reduces peripheral side effects.",
    "It should be taken twice daily on an empty stomach. Capsules must not be opened.",
    "People with urinary retention, moderate/severe liver problems, or untreated narrow-angle glaucoma should avoid it. It’s also contraindicated in those allergic to trospium.",
]

# Step 3: Loop through questions and get answers + contexts
answers = []
contexts = []

for question in questions:
    relevant_chunks = retrieve_relevant_chunks(question)
    gpt4_response = generate_gpt4_response(question, relevant_chunks)

    answers.append(gpt4_response)
    contexts.append(relevant_chunks)

# Step 4: Create dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "reference": ground_truths,
}

dataset = Dataset.from_dict(data)

# Step 5: Evaluate
result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

# Step 6: View results
df = result.to_pandas()
df

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What are the active ingredients in Cobenfy?,[Cobenfy ingredients \nActive ingredients: xa...,The active ingredients in Cobenfy are xanomeli...,Cobenfy contains xanomeline and trospium chlor...,1.0,1.0,1.00,1.0
1,How should Cobenfy be taken?,[about the health of women exposed to Cobenfy ...,Cobenfy should be taken as follows:\n\n- Take ...,It should be taken twice daily on an empty sto...,1.0,0.5,0.75,1.0
2,Who should not take Cobenfy?,[Who should not take this medicine?\nCobenfy s...,Cobenfy should not be taken by individuals who...,"People with urinary retention, moderate/severe...",1.0,1.0,1.00,1.0
